# Save RNA and ATAC data as anndata from transferred data

In [2]:
import anndata as ad
import scipy.io
import pandas as pd
import os
import anndata as ad

In [15]:
data_path = os.path.join("/lustre/groups/ml01/workspace/laura.martens/moretti_colab/archr_final", "Epicardiods", "Anndata")

In [8]:
with open(os.path.join(data_path, 'matrix.mtx'), 'rb') as mm:
    x = scipy.io.mmread(mm).T.tocsr()

In [11]:
obs = pd.read_csv(os.path.join(data_path, 'obs.csv'), sep=',')

In [13]:
var = pd.read_csv(os.path.join(data_path, 'var.csv'), sep=',')

In [19]:
var.index = var.seqnames + ":" + var.start.astype(str) + "-" + var.end.astype(str)

In [22]:
adata = ad.AnnData(X=x, obs=obs, var=var)
adata.var_names_make_unique()

/tmp/ipykernel_6232/1750927739.py:1: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = ad.AnnData(X=x, obs=obs, var=var)


In [26]:
adata.write(os.path.join(data_path, 'adata.h5ad'))

In [27]:
import scanpy as sc

In [28]:
sc.pp.filter_genes(adata, min_cells = int(adata.shape[0]*0.01))

In [30]:
adata.write(os.path.join(data_path, 'adata_filtered.h5ad'))

# Load RNA seq data

In [17]:
data_path = os.path.join("/lustre/groups/ml01/workspace/laura.martens/moretti_colab/transfer_data")

In [4]:
counts = pd.read_csv(os.path.join(data_path, "counts.csv"))

In [5]:
obs = pd.DataFrame(counts['clusters'], index = counts.index)

In [6]:
counts = counts.drop('clusters', axis =1)

In [7]:
var = pd.DataFrame(counts.columns.values, index=counts.columns.values, columns=["gene"])

In [8]:
adata = ad.AnnData(counts, obs=obs, var=var)

In [9]:
adata

AnnData object with n_obs × n_vars = 35499 × 24353
    obs: 'clusters'
    var: 'gene'

In [10]:
adata.obs['day'] = adata.obs_names.str.split('_').str[-1]

In [11]:
day_map = {
    '1':'2',
    '2': '3',
    '3': '4',
    '4': '5',
    '5': '7',
    '6': '10',
    '7': '15'
}

In [12]:
adata.obs['day'] = adata.obs.day.map(day_map)

In [18]:
adata.write(os.path.join(data_path, "Day2_15_RNAseq.h5ad"))

In [14]:
adata.obs

,clusters,day
AAACCCAAGAGCAAGA-1_5,22,7
AAACCCAAGCCGCACT-1_7,1,15
AAACCCAAGCCGTTGC-1_5,0,7
AAACCCAAGCGAGAAA-1_6,21,10
AAACCCAAGCTCCATA-1_1,2,2
...,...,...
TTTGTTGTCGGATACT-1_5,27,7
TTTGTTGTCGTCAAAC-1_4,8,5
TTTGTTGTCGTTCCCA-1_5,0,7
TTTGTTGTCTCTCTAA-1_6,6,10
